# News Articles Categorization

## Overview

 According to Internet Live Stats, in every second, around 9,000 tweets are posted; 3 million emails are sent and more than 90,000 GB of Internet traffic. So, we have a big amount of data in the internet. With such data, it is necessary to classify the data in categories. News contents are one type of data that should be categorized into groups to be easily accessed. News categorization helps users to access news of their interest without wasting time.
 
In this project,  I will apply machine learning algorithms on news collected from CNN News website to construct a model that classify the news into groups.

### Dataset


The dataset of this project from [kaggle](https://www.kaggle.com/c/learn-ai-bbc/data).

* BBC News Train.csv - The file contains a dataset of 1490 BBC News articles that classified into five categories: Business, Tech, Politics, Sport, Entertainment.

**Data Fields:**

   - ArticleId - Article id unique # given to the record.
   - Article - text of the header and article.
   - Category - cateogry of the article (tech, business, sport, entertainment, politics)


**The steps to build the model are:**
1. Data Exploration
2. Text Processing
3. Feature Extraction
4. Modeling
5. Use the Model


In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [1]:
# Import libraries

import pandas as pd
import re 

import nltk 
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost 
from sklearn.metrics  import classification_report
from sklearn import metrics
import time



## 1. Data Exploration

In [2]:
# Read the dataset from csv file
df1 = pd.read_csv('BBC News Training.csv')
df1.head()

ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business

In [3]:
# Find all category
category = list(df1['Category'].unique())
category

['business', 'tech', 'politics', 'sport', 'entertainment']

In [4]:
df1['Category'].value_counts().plot(kind='barh')
plt.show()

In [5]:
# Check null values
df1.isnull().sum()

ArticleId    0
Text         0
Category     0
dtype: int64

## 2. Text Preprocessing

In [6]:
# Text preprocessing
def preprocess(text):
    
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the article
    Return:
      lem(list of str): a list of the root form of the article words
    """
        
    # Normalize text
    text = re.sub(r"[^a-zA-Z]", " ", str(text).lower())
    
    # Tokenize text
    token = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in token if t not in stop]
    
    # Lemmatization
    lem = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lem

In [7]:
df1["Preprocessed_Text"] = df1['Text'].apply(lambda x: preprocess(x))
df1.head(10)

ArticleId                                               Text  \
0       1833  worldcom ex-boss launches defence lawyers defe...   
1        154  german business confidence slides german busin...   
2       1101  bbc poll indicates economic gloom citizens in ...   
3       1976  lifestyle  governs mobile choice  faster  bett...   
4        917  enron bosses in $168m payout eighteen former e...   
5       1582  howard  truanted to play snooker  conservative...   
6        651  wales silent on grand slam talk rhys williams ...   
7       1797  french honour for director parker british film...   
8       2034  car giant hit by mercedes slump a slump in pro...   
9       1866  fockers fuel festive film chart comedy meet th...   

        Category                                  Preprocessed_Text  
0       business  [worldcom, ex, bos, launch, defence, lawyer, d...  
1       business  [german, business, confidence, slide, german, ...  
2       business  [bbc, poll, indicates, economic, gloom, citize...  
3           tech  [lifestyle, governs, mobile, choice, faster, b...  
4       business  [enron, boss, payout, eighteen, former, enron,...  
5       politics  [howard, truanted, play, snooker, conservative...  
6          sport  [wale, silent, grand, slam, talk, rhys, willia...  
7  entertainment  [french, honour, director, parker, british, fi...  
8       business  [car, giant, hit, mercedes, slump, slump, prof...  
9  entertainment  [fockers, fuel, festive, film, chart, comedy, ...

## 2.1. Text Exploration

In [8]:
# Find the common words in each category
def find_common_words(df, category):
        
    """
    Function: find the most frequent words in the category and return the them
    Args:
      df(dataframe): the dataframe of articles
      category(str): the category name
    Return:
      the most frequant words in the category
    """
        
    # Create dataframes for the category
    cat_df = df[df["Category"]==category]
    
    # Initialize words list for the category
    words = [word for tokens in cat_df["Preprocessed_Text"] for word in tokens]
    
    # Count words frequency
    words_counter = Counter(words)
 
    return words_counter.most_common(10)

In [9]:
print("Most common words in each category")
for c in category:
    print(c, " News")
    print(find_common_words(df1, c))
    print()

Most common words in each category
business  News
[('said', 1100), ('year', 618), ('bn', 535), ('u', 523), ('mr', 394), ('company', 393), ('firm', 374), ('market', 346), ('would', 309), ('also', 279)]

tech  News
[('said', 1064), ('people', 647), ('mobile', 437), ('phone', 396), ('game', 396), ('technology', 380), ('service', 371), ('one', 367), ('year', 364), ('mr', 350)]

politics  News
[('said', 1445), ('mr', 1100), ('would', 712), ('labour', 494), ('election', 479), ('government', 471), ('party', 464), ('blair', 396), ('minister', 373), ('people', 372)]

sport  News
[('said', 636), ('game', 487), ('year', 448), ('first', 350), ('win', 337), ('time', 336), ('england', 329), ('player', 311), ('two', 290), ('back', 279)]

entertainment  News
[('film', 721), ('said', 594), ('year', 450), ('best', 430), ('award', 336), ('u', 290), ('also', 277), ('one', 274), ('show', 273), ('star', 259)]



In [10]:
df1['Preprocessed_Text2'] = df1['Preprocessed_Text'].apply(' '.join)
df1.head()

ArticleId                                               Text  Category  \
0       1833  worldcom ex-boss launches defence lawyers defe...  business   
1        154  german business confidence slides german busin...  business   
2       1101  bbc poll indicates economic gloom citizens in ...  business   
3       1976  lifestyle  governs mobile choice  faster  bett...      tech   
4        917  enron bosses in $168m payout eighteen former e...  business   

                                   Preprocessed_Text  \
0  [worldcom, ex, bos, launch, defence, lawyer, d...   
1  [german, business, confidence, slide, german, ...   
2  [bbc, poll, indicates, economic, gloom, citize...   
3  [lifestyle, governs, mobile, choice, faster, b...   
4  [enron, boss, payout, eighteen, former, enron,...   

                                  Preprocessed_Text2  
0  worldcom ex bos launch defence lawyer defendin...  
1  german business confidence slide german busine...  
2  bbc poll indicates economic gloom citizen majo...  
3  lifestyle governs mobile choice faster better ...  
4  enron boss payout eighteen former enron direct...

In [11]:
# Determine data and target
X = df1['Preprocessed_Text2']
y = df1['Category']

# 3. Feature Extraction

I will use TF-IDF method to extract the text features.

In [12]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [13]:
# Use TF-IDF
tf_vec = TfidfVectorizer()
train_features = tf_vec.fit(X_train)
train_features = tf_vec.transform(X_train)
test_features = tf_vec.transform(X_test)

# 4. Modeling

Here, I will use the below machine learning algorithms then I will select the best one based on its training time, accuracy score and f-beta scores. 

* Support Vector Machine 
* Ada Boost
* Gradient Boosting
* XG Boost
* Decision Tree
* Multinomial Naive Bayes 

In [14]:
# Train and evaluate model
def fit_eval_model(model, train_features, y_train, test_features, y_test):
    
    """
    Function: train and evaluate a machine learning classifier.
    Args:
      model: machine learning classifier
      train_features: train data extracted features
      y_train: train data lables
      test_features: train data extracted features
      y_test: train data lables
    Return:
      results(dictionary): a dictionary of the model training time and classification report
    """
    results ={}
    
    # Start time
    start = time.time()
    # Train the model
    model.fit(train_features, y_train)
    # End time
    end = time.time()
    # Calculate the training time
    results['train_time'] = end - start
    
    # Test the model
    train_predicted = model.predict(train_features)
    test_predicted = model.predict(test_features)
    
     # Classification report
    results['classification_report'] = classification_report(y_test, test_predicted)
        
    return results
    

In [15]:
# Initialize the models
sv = svm.SVC()
ab = AdaBoostClassifier(random_state = 1)
gb = GradientBoostingClassifier(random_state = 1)
xgb = xgboost.XGBClassifier(random_state = 1)
tree = DecisionTreeClassifier()
nb = MultinomialNB()


# Fit and evaluate models
results = {}
for cls in [sv, ab, gb, xgb, tree, nb]:
    cls_name = cls.__class__.__name__
    results[cls_name] = {}
    results[cls_name] = fit_eval_model(cls, train_features, y_train, test_features, y_test)
    

In [16]:
# Print classifiers results
for res in results:
    print (res)
    print()
    for i in results[res]:
        print (i, ':')
        print(results[res][i])
        print()
    print ('-----')
    print()

SVC

train_time :
6.316733121871948

classification_report :
               precision    recall  f1-score   support

     business       0.97      0.97      0.97        65
entertainment       0.98      0.98      0.98        61
     politics       0.98      0.98      0.98        51
        sport       0.97      1.00      0.99        70
         tech       0.98      0.94      0.96        51

     accuracy                           0.98       298
    macro avg       0.98      0.97      0.98       298
 weighted avg       0.98      0.98      0.98       298


-----

AdaBoostClassifier

train_time :
2.5002388954162598

classification_report :
               precision    recall  f1-score   support

     business       0.56      0.66      0.61        65
entertainment       0.85      0.77      0.81        61
     politics       0.72      0.71      0.71        51
        sport       0.78      0.90      0.83        70
         tech       0.89      0.61      0.72        51

     accuracy           

**Note:** From the testing results, we can see that **MultinomialNB** classifier is the fastest and most accurate classifier.

# 5. Use the Model

Now, I will use the built MultinomialNB model to classify new articles. The articles files inside *Articles folder* and here are the articles sources.  

* art1: https://edition.cnn.com/2019/09/30/sport/irish-national-stud-winning-post-spt-intl/index.html
* art2: https://edition.cnn.com/2020/04/15/tech/amazon-france-suspension/index.html
* art3: https://edition.cnn.com/2020/04/15/politics/barack-obama-2020-test/index.html
* art4: https://edition.cnn.com/2020/04/15/entertainment/disney-the-mandalorian-documentary/index.html


In [17]:
# Classify an article
def classify_article(path):
    
    """
    Function: classify an article.
    Args:
      path: the path of the article 
    Return:
      category (str): the category of the article
    """
    # Read file
    file = open(path, 'r')
    artcl = file.read()

    # Text preprocessing
    artcl = preprocess(artcl)
    artcl = ' '.join(artcl)

    # Use TF_IDF
    test = tf_vec.transform([artcl])

    # Use MultinomialNB model to classify the article
    predict = nb.predict(test)
    category = predict[0]

    # Close file
    file.close()

    return category

In [18]:
print(classify_article('Articles/art1.txt'))

sport


In [19]:
print(classify_article('Articles/art2.txt'))

business


In [20]:
print(classify_article('Articles/art3.txt'))

politics


In [21]:
print(classify_article('Articles/art4.txt'))

entertainment


**Note:** All articles classified correctly.  

# 6. Summary 

In this project, I used CNN news dataset to classify the news text. I used TF-IDF feature extraction algorithm and six different machine learning algorithms: Support Vector Machine , Ada Boost, Gradient Boosting, XG Boost, Decision Tree, Multinomial Naive Bayes. Multinomial Naive Bayes was the best model and then I use it to classify new articles in order to check the model accuracy. 